In [1]:
import numpy as np
import pandas as pd
import math as m
import cmath
import sympy as sym
import scipy as sci
import matplotlib.ticker as ticker
import random as rand
import matplotlib.pyplot as plt
import cmath as cm
from IPython.display import display, Latex
from sympy import pprint
from scipy.optimize import minimize as GDlib
import MLE_functions as fn
from scipy.optimize import fsolve
from scipy.optimize import least_squares

In [2]:
theta1, theta2, phi1, phi2, alpha = sym.symbols('theta1 theta2 phi1 phi2 alpha')
c1, c2, s1, s2, w1, w2 = sym.cos(theta1), sym.cos(theta2), sym.sin(theta1), sym.sin(theta2), sym.cos(alpha)**2, sym.sin(alpha)**2

# # analytic formuale for the probabilities, as given in the notes
pr1 = (1/6)*(w1*s1**2*(1+c1**2 -2*m.sqrt(2)*sym.cos(phi1)*s1*c1)+w2*s2**2*(1+c2**2-2*m.sqrt(2)*sym.cos(phi2)*s2*c2))
pr2 = (1/6)*(w1*c1**2*(1+s1**2 -2*m.sqrt(2)*sym.cos(phi1)*c1*s1) + w2*c2**2*(1+s2**2*-2*m.sqrt(2)*sym.cos(phi2)*c2*s2))
pr3 = (1/6)*(w1*(c1**4+s1**4 -2*sym.cos(2*phi1)*s1**2*c1**2)+ w2*(c2**4+ s2**4 - 2*sym.cos(2*phi2)*s2**2*c2**2))
pr4 = (1/6)*(w1*s1**2*(1+c1**2 -2*m.sqrt(2)*sym.cos(phi1+2*m.pi/3)*s1*c1)+ w2*s2**2*(1+c2**2-2*m.sqrt(2)*sym.cos(phi2+2*m.pi/3)*s2*c2))
# pr5= (1/6)* (w1*c1**2*(1+s1**2 -2*m.sqrt(2)*sym.cos(phi1-2*m.pi/3)*c1*s1)+ w2*c2**2*(1+s2**2-2*m.sqrt(2)*sym.cos(phi2-2*m.pi/3)*c2*s2))
pr5= (1/6)* (w1*c1**2*(1+s1**2 -2*m.sqrt(2)*sym.cos(phi1+2*m.pi/3)*c1*s1)+ w2*c2**2*(1+s2**2-2*m.sqrt(2)*sym.cos(phi2+2*m.pi/3)*c2*s2))
# pr6 = (1/6)*(w1*(c1**4+s1**4-2*sym.cos(2*phi1 - 2*m.pi/3)*c1**2*s1**2)+ w2*(c2**4+s2**4-2*sym.cos(2*phi2 - 2*m.pi/3)*c2**2*s2**2))
pr6 = (1/6)*(w1*(c1**4+s1**4-2*sym.cos(2*phi1 +2*m.pi/3)*c1**2*s1**2)+ w2*(c2**4+s2**4-2*sym.cos(2*phi2 +2*m.pi/3)*c2**2*s2**2))
pr7 = (1/6)*(w1*s1**2*(1+c1**2-2*m.sqrt(2)*sym.cos(phi1-2*m.pi/3)*s1*c1)+ w2*s2**2*(1+c2**2-2*m.sqrt(2)*sym.cos(phi2-2*m.pi/3)*s2*c2))
pr8 = (1/6)*(w1*c1**2*(1+s1**2-2*m.sqrt(2)*sym.cos(phi1-2*m.pi/3)*c1*s1)+ w2*c2**2*(1+s2**2-2*m.sqrt(2)*sym.cos(phi2-2*m.pi/3)*c2*s2))
pr9 = (1/6)*(w1*(c1**4+s1**4-2*sym.cos(2*phi1+2*m.pi/3)*c1**2*s1**2)+ w2*(c2**4+s2**4-2*sym.cos(2*phi2+2*m.pi/3)*c2**2*s2**2))
pr = [pr1, pr2,pr3, pr4,pr5, pr6,pr7, pr8,pr9]      # list of analytic prob.

w = m.e**((2/3)*m.pi*(1j))     # third root of unity
POVM_vec = (1/(2**.5))*(np.array([[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]))  # an array of POVM direction vectors
POVM_elts = [(1/3)*np.outer(np.conjugate(POVM_vec[i]),POVM_vec[i]) for i in range(len(POVM_vec))]   # a list of POVM matrices


In [3]:
pr1.subs({theta1:0, theta2:m.pi/2, phi1:0, phi2:0, alpha:m.pi/4})  

0.0833333333333333

In [4]:
def equations_th_prob(vars):                                        # equations are simply (p_j - n_j). p_j are the (substiuted) probabilities.
    t1, t2, p1, p2, a = vars[:5]
    # c1, c2, s1, s2, w1, w2 = sym.cos(theta1), sym.cos(theta2), sym.sin(theta1), sym.sin(theta2), sym.cos(alpha)**2, sym.sin(alpha)**2
    n_unnorm = [142, 128, 426, 2117, 1235, 1326, 2052, 1210, 1364]   # to be changed each time
    # correct_variables =  [m.pi/4, m.pi/2-m.pi/8 , 0,0, m.pi/4]     # the right answer, for the supposed n vector. just for tracking purposes
    n = [i/sum(n_unnorm) for i in n_unnorm]                          # normalise the counts
    
    eqs_pr = [i.subs({theta1: t1, theta2: t2, phi1: p1, phi2: p2 , alpha: a}).evalf() for i in pr]
    eqs    = [eqs_pr[i] - n[i] for i in range(len(n))]                  # final equations
    return eqs[4:]
    
equations_th_prob([0, m.pi/2, 0, 0, m.pi/4])   # just to check if the function is working

[-0.0401666666666667,
 0.0340666666666667,
 -0.121866666666667,
 -0.0376666666666667,
 0.0302666666666667]

In [5]:
pr_num = [sym.lambdify((theta1, theta2, phi1, phi2, alpha), i) for i in pr]   # lambdify the probabilities

def eq_pr_num(vars):
    n_unnorm = [142, 128, 426, 2117, 1235, 1326, 2052, 1210, 1364]   # to be changed each time
    n = [i/sum(n_unnorm) for i in n_unnorm]                          # normalise the counts
    eq_pr = [pr_num[i](*vars) for i in range(len(pr_num))]
    eq = [eq_pr[i] - n[i] for i in range(len(n))]
    return eq


In [6]:

inv_guess = [m.pi/4+.1, m.pi/2-m.pi/8 , 0,.001, m.pi/4]    # initial guess for the variables, and the rest are dummy zeros
solution = fsolve(equations_th_prob, inv_guess)
print('solution : ', solution)
print('sum of the absolute values of the residuals : ', sum([abs(i) for i in equations_th_prob(solution)]))


"""# solving using fsolve
sol_list = []
for i in range(10):
    initial_guess = list(np.random.rand(5))
    initial_guess = initial_guess + [0,0,0,0]
    solution = fsolve(equations_th_prob, initial_guess)
    sol_list.append(solution)

func_list = [sum([abs(j) for j in equations_th_prob(i)]) for i in sol_list]         # list for each solution, each term is the sum of the absolute values of (p_j-n_j)
print('','values : \n ', func_list)
"""



#solution :  [ 0.89819977  1.2252905  -0.18481207  0.30232177  0.59776286]
# sum of the absolute values of the residuals :  8.68815436261272e-12

solution :  [8.85398163e-01 1.17809725e+00 0.00000000e+00 1.00000000e-03
 7.85398163e-01]
sum of the absolute values of the residuals :  0.0541947243309272


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_minpack_py.py:177: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


"# solving using fsolve\nsol_list = []\nfor i in range(10):\n    initial_guess = list(np.random.rand(5))\n    initial_guess = initial_guess + [0,0,0,0]\n    solution = fsolve(equations_th_prob, initial_guess)\n    sol_list.append(solution)\n\nfunc_list = [sum([abs(j) for j in equations_th_prob(i)]) for i in sol_list]         # list for each solution, each term is the sum of the absolute values of (p_j-n_j)\nprint('','values : \n ', func_list)\n"

True solution : [0.7853981633974483, 1.1780972450961724, 0, 0, 0.7853981633974483]

In [7]:
# trying scipy least squares method
from scipy.optimize import least_squares
#creating loop over random initial guesses
sol_list = []
residue_list = []
for i in range(10):
    inv_guess = list(np.random.rand(5))
    result = least_squares(eq_pr_num, inv_guess)
    sol_list.append(result.x)
    residue_list.append(sum([abs(i) for i in result.fun]))

# print('solutions : ', sol_list)
print('residues : ', residue_list)
print('\n5 min residues : ', sorted(residue_list)[:5])    # the 5 smallest residues
print('\n \n last trial info', result)
print('\ntrue solution : ', [m.pi/4, m.pi/2-m.pi/8 , 0,0, m.pi/4])
print('\nfidelities', fn.fid(result.x, [m.pi/4, m.pi/2-m.pi/8 , 0,0, m.pi/4]))

residues :  [0.02729636688580301, 0.016092192472568267, 0.025683293530175524, 0.027296269101173068, 0.016092169177410777, 0.017271285036380705, 0.01727129048875934, 0.016092189463480032, 0.016092206423556847, 0.016092197534066585]

5 min residues :  [0.016092169177410777, 0.016092189463480032, 0.016092192472568267, 0.016092197534066585, 0.016092206423556847]

 
 last trial info      message: `gtol` termination condition is satisfied.
     success: True
      status: 1
         fun: [ 2.911e-03  2.219e-03 -3.379e-03  3.248e-04 -1.994e-03
                2.888e-03  7.708e-04  6.942e-04 -9.125e-04]
           x: [ 1.281e+00  8.534e-01  1.099e-01 -6.887e-03  9.731e-01]
        cost: 1.9570409020647932e-05
         jac: [[ 9.643e-02 -3.577e-02 ... -6.240e-04 -4.163e-02]
               [-2.069e-02 -8.070e-02 ... -2.697e-04 -8.430e-04]
               ...
               [-6.621e-02 -2.177e-01 ... -5.950e-02  1.243e-01]
               [ 1.563e-02  1.566e-02 ...  9.758e-02 -3.144e-02]]
        g

In [8]:
"""
# previous code for the loss function
# trying gradient descent method, but for the modified loss function. The new loss function is the absolute value of the difference between the probabilities and the counts

def loss_new(vars):                         
    # n_unnorm = [142, 128, 426, 2117, 1235, 1326, 2052, 1210, 1364]   # this is for params = [m.pi/4, m.pi/2-m.pi/8 , 0,0, m.pi/4]
    n_unnorm = [858, 820, 1701, 844, 794, 1616, 826, 825, 1716]     # this is for params = [0, m.pi/2, 0, 0, m.pi/4]
    
    n = [i/sum(n_unnorm) for i in n_unnorm]                          # normalise the counts
    eq_pr = [pr_num[i](*vars) for i in range(len(pr_num))]
    eq = [eq_pr[i] - n[i] for i in range(len(n))]
    loss = sum([abs(i) for i in eq])
    return loss
"""

'\n# previous code for the loss function\n# trying gradient descent method, but for the modified loss function. The new loss function is the absolute value of the difference between the probabilities and the counts\n\ndef loss_new(vars):                         \n    # n_unnorm = [142, 128, 426, 2117, 1235, 1326, 2052, 1210, 1364]   # this is for params = [m.pi/4, m.pi/2-m.pi/8 , 0,0, m.pi/4]\n    n_unnorm = [858, 820, 1701, 844, 794, 1616, 826, 825, 1716]     # this is for params = [0, m.pi/2, 0, 0, m.pi/4]\n    \n    n = [i/sum(n_unnorm) for i in n_unnorm]                          # normalise the counts\n    eq_pr = [pr_num[i](*vars) for i in range(len(pr_num))]\n    eq = [eq_pr[i] - n[i] for i in range(len(n))]\n    loss = sum([abs(i) for i in eq])\n    return loss\n'

In [9]:
"""#old loss
def loss_old(vars, params_i = [0, m.pi/2, 0, 0, m.pi/4], N = 10000, nj_unnorm = None):                         
    if nj_unnorm != None:
        n_unnorm = nj_unnorm
    else:
        num_expt = fn.num_experiment(params= params_i, N = N)              # for getting the collapse count
        n_unnorm = num_expt[4]

    n = [i/sum(n_unnorm) for i in n_unnorm]                          # normalise the counts
    eq_pr = [pr_num[i](*vars) for i in range(len(pr_num))]
    eq = [eq_pr[i] for i in range(len(n))]
    loss_old = sum([-i for i in eq])
    return loss_old
"""

'#old loss\ndef loss_old(vars, params_i = [0, m.pi/2, 0, 0, m.pi/4], N = 10000, nj_unnorm = None):                         \n    if nj_unnorm != None:\n        n_unnorm = nj_unnorm\n    else:\n        num_expt = fn.num_experiment(params= params_i, N = N)              # for getting the collapse count\n        n_unnorm = num_expt[4]\n\n    n = [i/sum(n_unnorm) for i in n_unnorm]                          # normalise the counts\n    eq_pr = [pr_num[i](*vars) for i in range(len(pr_num))]\n    eq = [eq_pr[i] for i in range(len(n))]\n    loss_old = sum([-i for i in eq])\n    return loss_old\n'

In [10]:
# Improving the previous loss function, new code: 
def loss_new(vars, params_tru = [0, m.pi/2, 0, 0, m.pi/4], N = 10000, nj_unnorm = None, lossfn = 'new'):                         
    if nj_unnorm != None:
        n_unnorm = nj_unnorm
    else:
        num_expt = fn.num_experiment(params= params_tru, N = N)                             # for getting the collapse count
        n_unnorm = num_expt[4]

    n = [i/sum(n_unnorm) for i in n_unnorm]                                                # normalise the counts
    eq_pr = [pr_num[i](*vars) for i in range(len(pr_num))]
    
    # eq = [eq_pr[i] - n[i] for i in range(len(n))]                                        # method of equating from the log, the probs and the counts
    lnl_p = [p * np.log(i) if i != 0 else 0 for i, p in zip(n, eq_pr)]                          # manually putting zero for cases of p_i*log(n_i) , if any n_i are 0.
    lnl_n = [i * np.log(i) if i != 0 else 0 for i in n]                         
    # eq = [n[i]*np.log(eq_pr[i]) - n[i]*np.log(n[i]) for i in range(len(n))]                # method of comparing the likelihood with the entropy/min likelihood
    del_lnl = [(k-l) for k,l in zip(lnl_p, lnl_n)]
    loss = sum([abs(i) for i in del_lnl])

    if lossfn == 'old':
        loss = sum([-(i) for i in lnl_p])
    xx = 5
    return loss


In [11]:
# initial_guess = list(np.random.rand(5))
inv_guess = [m.pi/4-.1, m.pi/2-m.pi/8-.1 , 0,.1, m.pi/4+ .4]    
GD_res = GDlib(loss_new, inv_guess)
print('result : ', GD_res)

print('',' \n \n initial guess', inv_guess)
print('','found solution : ', GD_res.x)
print('','true val', [m.pi/4, m.pi/2-m.pi/8 , 0,0, m.pi/4])
print('fidelities', fn.fid(GD_res.x, [m.pi/4, m.pi/2-m.pi/8 , 0,0, m.pi/4]))

result :    message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 1.4842637350202612
        x: [ 6.854e-01  1.078e+00  0.000e+00  1.000e-01  1.185e+00]
      nit: 0
      jac: [ 4.938e+05  6.306e+05  1.514e+06  2.804e+06  2.617e+06]
 hess_inv: [[1 0 ... 0 0]
            [0 1 ... 0 0]
            ...
            [0 0 ... 1 0]
            [0 0 ... 0 1]]
     nfev: 156
     njev: 24
  
 
 initial guess [0.6853981633974483, 1.0780972450961723, 0, 0.1, 1.1853981633974482]
 found solution :  [0.68539816 1.07809725 0.         0.1        1.18539816]
 true val [0.7853981633974483, 1.1780972450961724, 0, 0, 0.7853981633974483]
fidelities [0.9900332889206207, 0.98856109647552]


The hybride method seems to be working, when the new loss function is used instead of the old simple one. Exploring it further and after that creating graphs :

In [12]:
# idea: use inversion to get near, then run GDlib function. finally compare the fidelities.

def Hybrid(params_i = [0, m.pi/2, 0, 0, m.pi/4], N = 10000, threshold = 'variable'):
    inv = fn.Inversion(params = params_i, N = N, threshold = threshold)     # doing the inversion experiment
    fid_inv = inv[0]
    inv_params = inv[2]
    initial_guess = inv_params          # initial guess for the grad descent is the final answer of the inversion method, which is already very close 
    
    res_GD = GDlib(loss_new, initial_guess)   
    opt_params = res_GD.x
    fid = fn.fid(opt_params, params_i)          # fidelity of the final states with the sent states
    
    params_i_degrees =   [np.round(i*180/m.pi, 1) for i in params_i]
    inv_params_degrees = [np.round(i*180/m.pi, 1) for i in inv_params]   
    opt_params_degrees = [np.round(i*180/m.pi, 1) for i in opt_params]
    
    return fid


# Hybrid()   # just to check if the function is working
    



In [13]:
# idea: use inversion to get near, then run GDlib function. finally compare the fidelities.

def Hybrid(params_i = [0, m.pi/2, 0, 0, m.pi/4], N = 10000, threshold = 'variable'):
    inv = fn.Inversion(params = params_i, N = N, threshold = threshold)     # doing the inversion experiment
    fid_inv = inv[0]
    inv_params = inv[2]
    initial_guess = inv_params          # initial guess for the grad descent is the final answer of the inversion method, which is already very close 
    
    res_GD = GDlib(loss_new, initial_guess, args = (params_i, N))
    opt_params = res_GD.x
    fid = fn.fid(opt_params, params_i)          # fidelity of the final states with the sent states
    
    params_i_degrees =   [np.round(i*180/m.pi, 3) for i in params_i]
    inv_params_degrees = [np.round(i*180/m.pi, 3) for i in inv_params]   
    opt_params_degrees = [np.round(i*180/m.pi, 3) for i in opt_params]
    
    return fid


# Hybrid(params_i = [m.pi/8, m.pi/2-m.pi/8, 0, 0, m.pi/4], N = 10000, threshold = 'variable')   

In [26]:
#custom gradient descent algo, coz the scipy one doesnt converge
def grad(vars, params_i = None ,nj_unnorm = None, s=(1/500), N = 10000, loss = 'new'):    # "point" is the list of parameters
    if nj_unnorm != None:
        n_unnorm = nj_unnorm
    else:
        num_expt = fn.num_experiment(params= params_i, N = N)              # for getting the collapse count
        n_unnorm = num_expt[4]
    
    del_theta1 = loss_new(vars = [vars[0]+s,vars[1],vars[2],vars[3],vars[4]], nj_unnorm= nj_unnorm, lossfn = loss) - loss_new(vars = [vars[0],vars[1],vars[2],vars[3],vars[4]], nj_unnorm= nj_unnorm, lossfn = loss)
    del_theta2 = loss_new(vars = [vars[0],vars[1]+s,vars[2],vars[3],vars[4]], nj_unnorm= nj_unnorm, lossfn = loss) - loss_new(vars = [vars[0],vars[1],vars[2],vars[3],vars[4]], nj_unnorm= nj_unnorm, lossfn = loss)
    del_phi1   = loss_new(vars = [vars[0],vars[1],vars[2]+s,vars[3],vars[4]], nj_unnorm= nj_unnorm, lossfn = loss) - loss_new(vars = [vars[0],vars[1],vars[2],vars[3],vars[4]], nj_unnorm= nj_unnorm, lossfn = loss)
    del_phi2   = loss_new(vars = [vars[0],vars[1],vars[2],vars[3]+s,vars[4]], nj_unnorm= nj_unnorm, lossfn = loss) - loss_new(vars = [vars[0],vars[1],vars[2],vars[3],vars[4]], nj_unnorm= nj_unnorm, lossfn = loss)
    del_alpha  = loss_new(vars = [vars[0],vars[1],vars[2],vars[3],vars[4]+s], nj_unnorm= nj_unnorm, lossfn = loss) - loss_new(vars = [vars[0],vars[1],vars[2],vars[3],vars[4]], nj_unnorm= nj_unnorm, lossfn = loss)
    
    grad = [del_theta1/s,del_theta2/s,del_phi1/s,del_phi2/s,del_alpha/s]
    grad = [i if abs(i) > 1e-10 else 0 for i in grad]
    
    xx = 5
    return grad


# writing a gradient descent function for the new loss function

"""def GD(params_i = [0,m.pi/2,0,0,m.pi/4], nit = 20, step = 1/120, show_calc = False):
    
    grad_new = grad(vars = params_i, )
    params_new = p
    values = []
    for i in range(nit):
    grad_mod = (sum([abs(i)**2 for i in grad_new]))**(.5)
    params_new = [params_new[i]-s*(grad_new[i]/grad_mod) for i in range(len(params_i))]
    grad_new = grad(params_new)
    value = [loss_new(params_new), params_new, grad_new, i]
    values.append(value)
    if i > (3/10)*nit//1 and i > 10:
        s = 5*step
        if i > (8/10)*nit//1:
            s = 15*step
    if show_calc == True:
        print(value,'\n ')
return params_new"""

g = grad(vars = [0.7497958771991733, 0.9387189457887932, -0.03129290610777863, -3.135039348011299, (0.7039072158599036-3.0426163267295937e-18j)], nj_unnorm = [1323, 754, 0, 1289, 1311, 1342, 1321, 1363, 1297], loss = 'new')

KeyboardInterrupt: 

In [15]:
def GD_new(params_i, params_true = [0, m.pi/2, 0, 0, m.pi/4], nj_unnorm = None, nit = 20, step = 1/120, N = 10000, loss = 'new'):
    # if nj_unnorm != None:                                                           # priority is using the given collapses, else build using fn.num_expt
    #     nj_unnorm = nj_unnorm                                                                    
    # else:
    #     nj_unnorm = fn.num_experiment(params= params_true, N = N, an_pr= True)[4]           
    if nj_unnorm == None:                                                           # priority is using the given collapses, else build using fn.num_expt
        nj_unnorm = fn.num_experiment(params= params_true, N = N, an_pr= True)[4]           
    params = params_i
    loss_i = loss_new(vars = params, nj_unnorm= nj_unnorm)                                          # Initial parameters, loss
    
    params_history = [params]                                          # To store the history of parameters, loss and steps
    loss_history = [loss_i]                                         
    steps = [step]
    for i in range(nit):
        if loss == 'new':
            gradients = grad(vars = params, nj_unnorm= nj_unnorm)
        else:
            gradients = grad(vars = params, nj_unnorm= nj_unnorm, loss = 'old')
        params = [(i - step*grad) for i, grad in zip(params, gradients)]    # Basically, same as entry by entry updating the parameters.
        
        if i == (.5*nit)//1:                                   # variable step size after 50% and 75% of the iterations
            step = step/2
        if i == (.75*nit)//1:
            step = step/5
        steps.append(step)
        params_history.append(params)
        loss_history.append(loss_new(vars = params, nj_unnorm= nj_unnorm))
        if sum(abs(m-n) for m,n in zip(params, params_history[-2])) < 1e-10:
            print(f'Gradient descent converged at {params} after {i} iterations')
    xx = 6                                                          # dummy for debugging
    return params

params_tru1 = [m.pi/12, m.pi/2-m.pi/12, 0, 0, m.pi/4]        # 60 degrees, symmetric in 1st quadramt
# inv_guess = [0.2640432468429828, 1.3220290000177817, -0.09070684312662175, 0.0871477145803036, (0.7942549733540932)]

opt_GD = GD_new( params_i = np.array(params_tru1)+ np.array([.2,-.1,.3,.1,-.2]), params_true= params_tru1, nit = 20, step = 1/2, N = 10000, loss = 'new')

print('true params : ', params_tru1)
print('opt1 : ', opt_GD)
print('fidelity : ', fn.fid(opt_GD, params_tru1))


true params :  [0.2617993877991494, 1.3089969389957472, 0, 0, 0.7853981633974483]
opt1 :  [0.39144125708094557, 1.3393325856809397, 0.5964346298701403, 0.0840960428527506, 0.6244985013929705]
fidelity :  [0.9528421705964317, 0.9986854880047514]


In [16]:
# opt1 = GD_new([.2, m.pi/2+.3, .1, .3, m.pi/4+.2], nit = 30, step = 1/50, N = 10000)
opt_GD = GD_new([.2, m.pi/2+.3, .1, .3, m.pi/4+.2], [0, m.pi/2, 0,0,m.pi/4 ], nit = 30, step = 1/50, N = 10000)

print(fn.fid(opt_GD, [0, m.pi/2, 0, 0, m.pi/4]))

print(opt_GD)
print([0, m.pi/2, 0, 0, m.pi/4])


[0.9999923064378412, 0.9999739958697293]
[0.002773730680720482, 1.5758957734000354, 0.07982622120976857, 0.2738479925024069, 0.7828944726658738]
[0, 1.5707963267948966, 0, 0, 0.7853981633974483]


In [17]:
# the new GD works. Trying more examples, comparing with the inversion method results
#MAIN RESULTS
params_tru1 = [m.pi/8, m.pi/2-m.pi/8, 0, 0, m.pi/4]        # 45 degrees, symmetric in 1st quadramt
a = [rand.uniform(0,m.pi/2) for i in range(5)]


opt_GD = GD_new( params_i = np.array(params_tru1)+ np.array([.2,-.1,.3,.1,-.2]), params_true= [m.pi/8, m.pi/2-m.pi/8, 0, 0, m.pi/4], nit = 100, step = 1/70, N = 10000)
inv = fn.Inversion_new(params = params_tru1, N = 10000, threshold = 'variable')
opt_inv = inv[2]
fid_inv = inv[0]

print('**************')
print(f'opt1 : {opt_GD} \n tru: {params_tru1} \n inv_opt1 : {opt_inv} \n fid : {fn.fid(opt_GD, params_tru1)} \n inv_fid : {fid_inv}')

**************
opt1 : [0.3924635740902881, 1.1593026216706654, 0.09321136790956905, -0.01055517992478513, 0.7970449854139229] 
 tru: [0.39269908169872414, 1.1780972450961724, 0, 0, 0.7853981633974483] 
 inv_opt1 : [0.42894315024735835, 1.205053566080319, -0.10281952589940356, -0.06935898188890478, (0.7393841575881271-1.3161585254031097e-17j)] 
 fid : [0.9989151970427949, 0.9996323638518374] 
 inv_fid : [0.9972744682446901, 0.9987057043660503]


In [18]:
params_tru1 = [m.pi/4, m.pi/2+m.pi/4, 0, 0, m.pi/4]        # 45 degrees around +y axis

opt_GD = GD_new( params_i = np.array(params_tru1)+ np.array([.2,-.1,.3,.1,-.2]), params_true= [m.pi/8, m.pi/2-m.pi/8, 0, 0, m.pi/4], nit = 150, step = 1/150, N = 10000)
inv = fn.Inversion_new(params = params_tru1, N = 10000, threshold = 'variable')
opt_inv = inv[2]
fid_inv = inv[0]

print('**************')
print(f'opt1 : {opt_GD} \n tru: {params_tru1} \n inv_opt1 : {opt_inv} \n fid : {fn.fid(opt_GD, params_tru1)} \n inv_fid : {fid_inv}')

**************
opt1 : [0.8307122743238626, 2.2649587870653263, 0.2289091344452977, 0.10277245437261867, 0.24262332055383792] 
 tru: [0.7853981633974483, 2.356194490192345, 0, 0, 0.7853981633974483] 
 inv_opt1 : [0.764968395048091, 0.9210267917186483, 0.0074979798787395, -3.1147480592765593, (0.7130394459296733-1.9306544924011948e-18j)] 
 fid : [0.9849588171152427, 0.9891046950479263] 
 inv_fid : [0.9995686395003334, 0.9815438323155526]


In [19]:
params_tru1 = [m.pi/4, m.pi/2+m.pi/4, 0, 0, m.pi/8]     # smaller alpha


opt_GD = GD_new( params_i = np.array(params_tru1)+ np.array([.2,-.1,.3,.1,-.2]), params_true= [m.pi/8, m.pi/2-m.pi/8, 0, 0, m.pi/4], nit = 100, step = 1/30, N = 10000)
inv = fn.Inversion_new(params = params_tru1, N = 10000, threshold = 'variable')
opt_inv = inv[2]
fid_inv = inv[0]

print('**************')
print(f'opt1 : {opt_GD} \n tru: {params_tru1} \n inv_opt1 : {opt_inv} \n fid : {fn.fid(opt_GD, params_tru1)} \n inv_fid : {fid_inv}')

**************
opt1 : [0.7332149883677442, 2.2453262257391087, 0.1751708024020456, 0.09599787964324194, 0.20357026537777298] 
 tru: [0.7853981633974483, 2.356194490192345, 0, 0, 0.39269908169872414] 
 inv_opt1 : [0.795722429049192, 0.9515225785338458, 0.013511744947243418, -3.101715467918529, (0.35114419683630715-8.704203544984617e-18j)] 
 fid : [0.9896694144220264, 0.9855127415703802] 
 inv_fid : [0.9998477819368514, 0.9722798607902248]


In [20]:
params_tru1 = [m.pi/12, m.pi/2-m.pi/12, 0, 0, m.pi/4]        # 60 degrees, symmetric in 1st quadramt
# inv_guess = [0.2640432468429828, 1.3220290000177817, -0.09070684312662175, 0.0871477145803036, (0.7942549733540932)]

opt_GD = GD_new( params_i = np.array(params_tru1)+ np.array([.2,-.1,.3,.1,-.2]), params_true= [m.pi/8, m.pi/2-m.pi/8, 0, 0, m.pi/4], nit = 1000, step = 1/10, N = 10000, loss = 'new')
# opt1 = GD_new( params_i = inv_guess , params_true= [m.pi/8, m.pi/2-m.pi/8, 0, 0, m.pi/4], nit = 5, step = 1/100, N = 10000)
inv = fn.Inversion_new(params = params_tru1, N = 10000, threshold = 'variable')
opt_inv = inv[2]
fid_inv = inv[0]

print('**************')
print(f'\n opt1 : {opt_GD} \n tru: {params_tru1} \n inv_opt1 : {opt_inv} \n fid : {fn.fid(opt_GD, params_tru1)} \n inv_fid : {fid_inv}')
print('\n**************')

**************

 opt1 : [0.3637451682143003, 1.1848520808896752, 0.022800422273176593, 0.005606011675520266, 0.8049902701566931] 
 tru: [0.2617993877991494, 1.3089969389957472, 0, 0, 0.7853981633974483] 
 inv_opt1 : [0.27718954079343294, 1.3165191283194537, 0.04485633761689563, -0.05807442691937226, (0.7742191390949461+5.930828419406237e-18j)] 
 fid : [0.9895998011561535, 0.9846643276821502] 
 inv_fid : [0.999630784606342, 0.9997382018126175]

**************


In [21]:
params_tru1 = [m.pi/12, m.pi/2-m.pi/12, 0, 0, m.pi/4]        # 60 degrees, symmetric in 1st quadramt
# inv_guess = [0.2640432468429828, 1.3220290000177817, -0.09070684312662175, 0.0871477145803036, (0.7942549733540932)]

opt_GD = GD_new( params_i = np.array(params_tru1)+ np.array([.2,-.1,.3,.1,-.2]), params_true= [m.pi/8, m.pi/2-m.pi/8, 0, 0, m.pi/4], nit = 10, step = 1/10, N = 10000, loss = 'new')


In [22]:
"""#rough
params_tru1 = [m.pi/12, m.pi/2-m.pi/12, 0, 0, m.pi/4]  
res = GDlib(loss_new, np.array(params_tru1)+ np.array([.2,-.1,.3,.1,-.2]), args = (params_tru1, 10000, None, 'new'), method='Nelder-Mead')
print(res)

#fidelities
fid= fn.fid(res.x, params_tru1)
print('fid',fid)"""

"#rough\nparams_tru1 = [m.pi/12, m.pi/2-m.pi/12, 0, 0, m.pi/4]  \nres = GDlib(loss_new, np.array(params_tru1)+ np.array([.2,-.1,.3,.1,-.2]), args = (params_tru1, 10000, None, 'new'), method='Nelder-Mead')\nprint(res)\n\n#fidelities\nfid= fn.fid(res.x, params_tru1)\nprint('fid',fid)"

In [29]:
#Creating table of results. Comparing the results of the inversion method and the gradient descent method (using the new loss function)

# true value = [m.pi/8, m.pi/2-m.pi/8, 0, 0, m.pi/4]
params_tru = [m.pi/8, m.pi/2-m.pi/8, 0, 0, m.pi/4]

def results(params_tru, nit = 100, step = 1/70, N = 10000):
    inv = fn.Inversion_new(params = params_tru, N = N, threshold = 'variable')
    nj_unnorm = inv[6]
    opt_inv = inv[2]
    # inv_fid_c = inv[0]                              # using coeffcients from the inversion protocol to calculate fidelity
    fid_inv = fn.fid(opt_inv, params_tru)        # using parameters to calculate fidelity, should be same as from the coefficients
    
    opt_GD = GD_new( params_i = opt_inv , nj_unnorm = nj_unnorm, nit = nit, step = step, N = N)
    fid_opt = fn.fid(opt_GD, params_tru)
    
    ppm_inv_fid = [(1- i)*1e6 for i in fid_inv]     # parts per million error in fids
    ppm_opt_fid = [(1- i)*1e6 for i in fid_opt]
    print(f'tru_params_deg: {[i*(180/m.pi) for i in params_tru]} \n tru_params: {params_tru} \n opt_inv: {opt_inv} \n opt_GD: {opt_GD} \n \n  inv_fid: {fid_inv} \n GD_fid: {fid_opt}  \n\n ppm_inv_fid: {ppm_inv_fid} \n ppm_opt_fid: {ppm_opt_fid} \n')

    return [params_tru, opt_inv, opt_GD , fid_inv, fid_opt]


Examples for sending to prof


In [35]:
params_tru1 = [m.pi/12, m.pi/2-m.pi/12, 0, 0, m.pi/4]   # 60 degrees, symmetric in 1st quadrant

results(params_tru1, nit = 200, step = 1/10, N = 10000)

tru_params_deg: [14.999999999999998, 75.0, 0.0, 0.0, 45.0] 
 tru_params: [0.2617993877991494, 1.3089969389957472, 0, 0, 0.7853981633974483] 
 opt_inv: [0.26822093903074223, 1.3152268580213222, -0.0998557951966819, -0.07997179870362393, (0.7855608729803474+1.5907079090525755e-18j)] 
 opt_GD: [0.24276559152604382, 1.3133280465128043, -0.06366771380786686, -0.039137620112997054, (0.7885650955912604+1.5907079090525755e-18j)] 
 
  inv_fid: [0.9993222834226456, 0.999570336096648] 
 GD_fid: [0.9994013736571075, 0.9998869589965258]  

 ppm_inv_fid: [677.7165773543813, 429.6639033519734] 
 ppm_opt_fid: [598.6263428925387, 113.04100347420132] 



[[0.2617993877991494, 1.3089969389957472, 0, 0, 0.7853981633974483],
 [0.26822093903074223,
  1.3152268580213222,
  -0.0998557951966819,
  -0.07997179870362393,
  (0.7855608729803474+1.5907079090525755e-18j)],
 [0.24276559152604382,
  1.3133280465128043,
  -0.06366771380786686,
  -0.039137620112997054,
  (0.7885650955912604+1.5907079090525755e-18j)],
 [0.9993222834226456, 0.999570336096648],
 [0.9994013736571075, 0.9998869589965258]]

In [32]:
params_tru1 = [m.pi/4, m.pi/2+m.pi/4, 0, 0, m.pi/4]   # 90 degrees, symmetric to +y axis

results(params_tru1, nit = 100, step = 1/4, N = 10000)

tru_params_deg: [45.0, 135.0, 0.0, 0.0, 45.0] 
 tru_params: [0.7853981633974483, 2.356194490192345, 0, 0, 0.7853981633974483] 
 opt_inv: [0.760436241312643, 0.9225742255817486, -0.029157606400135486, 3.122604226403839, (0.7173023200024781-3.260824506144435e-18j)] 
 opt_GD: [0.7782118509303689, 0.8149008493385945, 0.028610590468963373, 3.1921349506973336, (0.7753834437750255-3.260824506144435e-18j)] 
 
  inv_fid: [0.9991647702011369, 0.9812136958082733] 
 GD_fid: [0.9997437514242019, 0.9984924601994433]  

 ppm_inv_fid: [835.2297988630797, 18786.304191726667] 
 ppm_opt_fid: [256.24857579809037, 1507.5398005567342] 



[[0.7853981633974483, 2.356194490192345, 0, 0, 0.7853981633974483],
 [0.760436241312643,
  0.9225742255817486,
  -0.029157606400135486,
  3.122604226403839,
  (0.7173023200024781-3.260824506144435e-18j)],
 [0.7782118509303689,
  0.8149008493385945,
  0.028610590468963373,
  3.1921349506973336,
  (0.7753834437750255-3.260824506144435e-18j)],
 [0.9991647702011369, 0.9812136958082733],
 [0.9997437514242019, 0.9984924601994433]]

In [37]:
# 90 deg : (30, 120 deg)
params_tru1 = [m.pi/6, m.pi/2+m.pi/6, 0, 0, m.pi/4]   

results(params_tru1, nit = 1000, step = 1/20, N = 10000)

tru_params_deg: [29.999999999999996, 119.99999999999999, 0.0, 0.0, 45.0] 
 tru_params: [0.5235987755982988, 2.0943951023931953, 0, 0, 0.7853981633974483] 
 opt_inv: [0.5466839134667312, 1.1137474542780048, -0.02610471231505182, -3.113696061516407, (0.7600429759468649+6.383284426249612e-20j)] 
 opt_GD: [0.5412658612637588, 1.0668487939934668, -0.011217062705440345, -3.0935770884404192, (0.7748952389521798+6.383284426249612e-20j)] 
 
  inv_fid: [0.9993361369138606, 0.9954442077978668] 
 GD_fid: [0.99966384864141, 0.9991918195432765]  

 ppm_inv_fid: [663.8630861394157, 4555.792202133202] 
 ppm_opt_fid: [336.1513585899978, 808.180456723484] 



[[0.5235987755982988, 2.0943951023931953, 0, 0, 0.7853981633974483],
 [0.5466839134667312,
  1.1137474542780048,
  -0.02610471231505182,
  -3.113696061516407,
  (0.7600429759468649+6.383284426249612e-20j)],
 [0.5412658612637588,
  1.0668487939934668,
  -0.011217062705440345,
  -3.0935770884404192,
  (0.7748952389521798+6.383284426249612e-20j)],
 [0.9993361369138606, 0.9954442077978668],
 [0.99966384864141, 0.9991918195432765]]

In [39]:
# 90 deg : (30, 120 deg); this time smaller alpha
params_tru1 = [m.pi/6, m.pi/2+m.pi/6, 0, 0, m.pi/6]

results(params_tru1, nit = 200, step = 1/10, N = 10000)

tru_params_deg: [29.999999999999996, 119.99999999999999, 0.0, 0.0, 29.999999999999996] 
 tru_params: [0.5235987755982988, 2.0943951023931953, 0, 0, 0.5235987755982988] 
 opt_inv: [0.520485416640111, 1.1118719934403491, 0.048973286483934945, -3.068867696543849, (0.4867482799867063+4.152601949231743e-18j)] 
 opt_GD: [0.5099336325957732, 1.067218082434157, 0.02826232632797275, -3.141224862932065, (0.5142003694927895+4.152601949231743e-18j)] 
 
  inv_fid: [0.9995423251671268, 0.9949139092924759] 
 GD_fid: [0.9996659368054794, 0.9995992071221663]  

 ppm_inv_fid: [457.67483287317833, 5086.090707524082] 
 ppm_opt_fid: [334.06319452056186, 400.7928778336778] 



[[0.5235987755982988, 2.0943951023931953, 0, 0, 0.5235987755982988],
 [0.520485416640111,
  1.1118719934403491,
  0.048973286483934945,
  -3.068867696543849,
  (0.4867482799867063+4.152601949231743e-18j)],
 [0.5099336325957732,
  1.067218082434157,
  0.02826232632797275,
  -3.141224862932065,
  (0.5142003694927895+4.152601949231743e-18j)],
 [0.9995423251671268, 0.9949139092924759],
 [0.9996659368054794, 0.9995992071221663]]

In [44]:
# 90 deg : (30, 120 deg). smaller alpha AND random phi angles.
params_tru1 = [m.pi/6, m.pi/2+m.pi/6, m.pi/4, m.pi/2, m.pi/6]
results(params_tru1, nit = 500, step = 1/10, N = 10000)

tru_params_deg: [29.999999999999996, 119.99999999999999, 45.0, 90.0, 29.999999999999996] 
 tru_params: [0.5235987755982988, 2.0943951023931953, 0.7853981633974483, 1.5707963267948966, 0.5235987755982988] 
 opt_inv: [0.4682041048878335, 1.3693299151594398, 0.7692265173690841, -0.3528863562497126, (0.6232648008274801-1.6179898358444556e-17j)] 
 opt_GD: [0.48487692940446503, 1.1092739184859342, 0.8092943259889571, -1.307371119859963, (0.5930335042277685-1.6179898358444556e-17j)] 
 
  inv_fid: [0.9968889642026643, 0.7886601858599687] 
 GD_fid: [0.9983994085677436, 0.9842398242673239]  

 ppm_inv_fid: [3111.0357973357372, 211339.81414003135] 
 ppm_opt_fid: [1600.5914322564374, 15760.175732676074] 



[[0.5235987755982988,
  2.0943951023931953,
  0.7853981633974483,
  1.5707963267948966,
  0.5235987755982988],
 [0.4682041048878335,
  1.3693299151594398,
  0.7692265173690841,
  -0.3528863562497126,
  (0.6232648008274801-1.6179898358444556e-17j)],
 [0.48487692940446503,
  1.1092739184859342,
  0.8092943259889571,
  -1.307371119859963,
  (0.5930335042277685-1.6179898358444556e-17j)],
 [0.9968889642026643, 0.7886601858599687],
 [0.9983994085677436, 0.9842398242673239]]